In [15]:
from MySqlHelper import MySqlHelper
helper = MySqlHelper(host='localhost', user='root', password='@Kevin021117', database='school')
helper.connect()


In [16]:
create_sql = """
CREATE TABLE IF NOT EXISTS student (
    id INT PRIMARY KEY,
    name VARCHAR(50),
    height DECIMAL(5,2)
);
"""
helper.execute(create_sql)


0

In [17]:
create_sql

'\nCREATE TABLE IF NOT EXISTS student (\n    id INT PRIMARY KEY,\n    name VARCHAR(50),\n    height DECIMAL(5,2)\n);\n'

In [18]:
insert_sql = "INSERT INTO student (id, name, height) VALUES (%s, %s, %s)"
helper.execute(insert_sql, (3, 'aaa', 175.50))
helper.execute(insert_sql, (4, 'bbb', 168.00))


执行失败，正在回滚：(1062, "Duplicate entry '3' for key 'student.PRIMARY'")
执行失败，正在回滚：(1062, "Duplicate entry '4' for key 'student.PRIMARY'")


0

In [19]:
select_all = "SELECT * FROM student"
students = helper.query(select_all)
print(students)



[{'id': 1, 'name': '张三', 'height': Decimal('180.00')}, {'id': 3, 'name': 'aaa', 'height': Decimal('175.50')}, {'id': 4, 'name': 'bbb', 'height': Decimal('187.00')}]


In [20]:
update_sql = "UPDATE student SET height = %s WHERE id = %s"
helper.execute(update_sql, (187.00, 4))


0

In [21]:
delete_sql = "DELETE FROM student WHERE id = %s"
helper.execute(delete_sql, (2,))

0

In [22]:
select_all = "SELECT * FROM student"
students = helper.query(select_all)
print(students)

[{'id': 1, 'name': '张三', 'height': Decimal('180.00')}, {'id': 3, 'name': 'aaa', 'height': Decimal('175.50')}, {'id': 4, 'name': 'bbb', 'height': Decimal('187.00')}]
